# HBC_TSY_ENCHANCED

# As Jobs

## Pipeline (load/transform/persist)

In [ ]:
from pathlib import Path
import tempfile

dir_base = Path(tempfile.gettempdir()) / "HBC_TASK_BASE_FOLDER"
dir_analytics = dir_base / "ANALYTICS"
dir_logging = dir_base / "LOGGING"

%cd -q ..
!python -m hbc.jobs.dispatch \
  --job-name=job_fetch_nyc_open_data_311_service_requests \
  --as-of=20091231 \
  --dir-base="{dir_base}" \
  --dir-analytics="{dir_analytics}" \
  --dir-logging="{dir_logging}" \
  --incremental=True \
  --log-level=DEBUG
%cd -q notebooks/


In [ ]:
# restore cache integrity for the last missing dates
%cd -q ..
!python -m hbc.jobs.dispatch  \
      --job-name=job_fetch_nyc_open_data_311_service_requests \
      --as-of=20091231 \
      --incremental=false \
      --log-level=INFO \
      --last-missing-dates=10
%cd -q notebooks/

## Analytics

In [ ]:
%cd -q ..
!python -m hbc.jobs.dispatch  \
      --job-name=job_analyse_nyc_open_data_311_service_requests \
      --as-of=20091231 \
      --log-level=INFO \
      --n-worst=10 \
      --n-best=10 \
      --n-days=10
%cd -q notebooks/

***

# As Library 

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Imports

In [ ]:
import os
import sys
from pathlib import Path

In [ ]:
p = str(Path.cwd().parent) # one dir up
if p not in sys.path:
    sys.path.insert(0, p)

import pandas as pd
import os
import numpy as np
import logging
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

## Api

In [ ]:
from hbc import app_context, DataContainer, utils as ul
from hbc.quant.analysis import AnalyticalEngine
from hbc.quant.plots import PlotEngine

In [ ]:
app_context

## Logging

In [ ]:
import logging
logger = logging.getLogger()
ul.conf_log(level=logging.INFO, console=True, file=True, reset_handlers=True)

## Data Container : nyc_open_data_311_customer_satisfaction_survey

In [ ]:
dc = DataContainer("nyc_open_data_311_customer_satisfaction_survey")

In [ ]:
# retrieve: query first 100 rows
dc.get()
dc.df.shape

In [ ]:
# retrieve: query distinct
dc.get(query="$apply=groupby((campaign))")
dc.df.shape

In [ ]:
# retrieve: query with filter
dc.get(query="$filter=campaign eq 'Campaign 4'")
dc.df.shape

In [ ]:
# caching: 
dc.to_cache()

In [ ]:
# from_cache: get 100 rows
dc.from_cache()
dc.df.shape

In [ ]:
# from_cache: get by filter
dc.from_cache(query="$filter=campaign eq 'Campaign 4'")
dc.df.shape

In [ ]:
# from_cache: get distinct
dc.from_cache(query="$apply=groupby((year))")
dc.df.shape

In [ ]:
# from_cache: get page 2 with page size 50
dc.from_cache(query="$top=50&$skip=50")
dc.df.shape

In [ ]:
# from_cache: get total count
dc.from_cache(query="$count=true")
dc.df.shape

## Analytics

In [ ]:
app_context.as_of  = ul.str_as_date('20091231')

In [ ]:
dc = DataContainer('nyc_open_data_311_service_requests')

In [ ]:
# query / load / validate:
dc.get(query=f"$filter=created_date eq '{ul.date_as_iso_format(app_context.as_of)}'")

In [ ]:
# persist:
dc.to_cache()

In [ ]:
# retrieve from cache for analytics:
dc.from_cache(query=f"$filter=created_date eq '{ul.date_as_iso_format(app_context.as_of)}'")

In [ ]:
df = dc.df

In [ ]:
cols = ul.cols_as_named_tuple(df)

In [ ]:
df["hbc_days_to_close"] = (
    pd.to_datetime(df[cols.closed_date])
    - pd.to_datetime(df[cols.created_date])
).dt.days.astype("Int64")
cols = ul.cols_as_named_tuple(df)

In [ ]:
m = df[cols.hbc_days_to_close] == 0
df_closed_not_same_day = df[~m]

In [ ]:
path = ul.path_to_str(
                ul.mk_dir(app_context.dir_analytics / "plots")
                / "closed_requests_by_location.html"
            )
_ = PlotEngine.plot_geo_map(
            df=df_closed_not_same_day,
            col_latitude=cols.latitude,
            col_longitude=cols.longitude,
            aggregation="count",
            round_precision=3,
            cluster=True,
            start_zoom=11,
            tiles="CartoDB positron",
            savepath= path
        )
print(path)

In [ ]:
# by agency
res = AnalyticalEngine.descriptive_stats(
    n_best=10,
    n_worst=10,
    df=df_closed_not_same_day,
    col_metric=cols.hbc_days_to_close,
    group=[
        cols.agency,
        cols.agency_name,
    ],
)


In [ ]:
res.keys()

In [ ]:
res['worst']

## Data Container : nyc_open_data_311_call_center_inquiry

In [ ]:
dc = DataContainer('nyc_open_data_311_call_center_inquiry')

In [ ]:
# load / validate:
dc.get()
dc.df.head(2)

In [ ]:
# persist
dc.to_cache()

In [ ]:
# query at loading:
dc.get(query="$filter=agency eq 'NYPD' and date eq '2014-03-27'&$top=250")
dc.df.head(2)

In [ ]:
# query at caching:
dc.from_cache(query="$filter=date eq '2014-03-27'")
dc.df.head(2)

## Data Container : nyc_open_data_311_customer_satisfaction_survey

In [ ]:
dc = DataContainer('nyc_open_data_311_customer_satisfaction_survey')

In [ ]:
# load / validate:
dc.get()
dc.df.head(2)

In [ ]:
# cache:
dc.to_cache()

In [ ]:
# query from caching:
dc.from_cache(query="$filter=answer_satisfaction eq 'Neutral'")
dc.df.head(2)

***

## `Raw DB Access`

In [ ]:
from hbc.ltp.persistence.db import SqlLiteDataBase

In [ ]:
db = SqlLiteDataBase()
print([db.run_query(f'select count(*) from {t}') for t in db.all_tables])
db.close()

## `Files/folders-based cache`

In [ ]:
from hbc.ltp.persistence.cache import Cache

In [ ]:
dc = DataContainer('nyc_open_data_311_customer_satisfaction_survey')

In [ ]:
dc.get(query='$top=10')

In [ ]:
Cache.to_cache(dc, as_of=app_context.as_of)

In [ ]:
Cache.from_cache(dc, as_of=app_context.as_of).head(2)